In [ ]:
from _util import *

import _RL.FQI as FQI
import _RL.my_gym as my_gym
import _Ohio_Simulator as Ohio
import _analyze as analyze
import _RL.DQN as DQN
import _cartpole as cartpole
from coinDice import run_neural_coin_dice as run_coin
from coinDice import converter as converter

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"

reload(converter)
reload(run_coin)

1. we can either store the data generated from TR, or re-generate it 

## N10, T = 300, tau = 0.3, algae_alpha = 0.05

In [2]:
###################################################################################################################
unique_setting = "cart_coindice"
n_gpu = 8
rep = 200
####
tau = 0.3
N = 10
T = 300
####
########################################################################
gamma = .98
env = cartpole.CartPoleEnv(e_max = 1000, std = 0.01)
setting_no_time = unique_setting + "_" + "N" + str(N) + "_" + "T" + str(T) +  "_" + "tau" + "_" + str(tau)
setting = EST()[7:9] + EST()[10:13]+ "_"  + setting_no_time
printR("setting: {}".format(setting))
############ 

data_path = "data/" + "1103_cart_T300_Q_N" + str(N) + "_T300_initQ0.99_tau_" + str(tau) + "Qs"
##################################################################################################################
weight_path = "target_policies/" + "cartpole" + "g" + str(0.99)
tp_path = weight_path + "/iter" + str(20000)

pi1 = DQN.DQN_gym(num_states = 4, num_actions = 2
                  , hidden_units = [256, 256], gamma = gamma
                  , gpu_number = 0)
pi1.model.load_weights(tp_path)
#########################################################
pi_behav = my_gym.softmax_policy(pi1, tau)

gym_eval = my_gym.GymEval(random_pi = True)

try:
    cart_data = load(data_path)
    trajs_train_resp = cart_data["trajs_train_resp"]
    V_true = cart_data["V_true"]
except:
    pass
    # cart_data = {
    #     "training" : trajs_train_resp
    #     , "V_true" : V_true
    # }
    # dump(cart_data, "data/" + setting_no_time)

###################################################################################################################
tf.keras.backend.set_floatx('float32')
printR("V_true = {}".format(V_true))

setting: 1105_cart_coindice_N10_T300_tau_0.3
V_true = 44.62911337541588


In [ ]:
rep = 200
alpha_CI = 0.9
###########
lr = 0.005
batch_size = 32
algae_alpha = 0.05
ray.shutdown()
a = now()
res_all = []
for i in range(rep // n_gpu):
    @ray.remote(num_gpus=1)
    def one_seed1(seed):
        gpu_idx = seed % n_gpu
        pi1 = DQN.DQN_gym(num_states = 4, num_actions = 2
                          , hidden_units = [256, 256], gamma = gamma
                          , gpu_number = gpu_idx)
        pi1.model.load_weights(tp_path)
        dataset = converter.convertDataset(trajs_train_resp[seed]
                                      , min_A = 0, max_A = 1
                                       , min_S = -5, max_S = 5)
        CI = run_coin.coindice(dataset = dataset, pi1 = pi1, N = N, T = T, gamma = gamma
                      , hidden_dims = (64, ), gpu_number = gpu_idx
                 , nu_learning_rate = lr, zeta_learning_rate = lr, weight_learning_rate = lr ### ????
                 , nu_regularizer = 1, zeta_regularizer = 1 # NOT SENSITIVE
                 , num_steps = 2000, batch_size = batch_size, alpha_CI = alpha_CI
                 , print_loss = 0, print_freq = 50, print_prog = 0
                 , primal_form = True, f_exponent = 1.5, algae_alpha = algae_alpha)
        return CI
    ray.init()
    ###########
    futures = [one_seed1.remote(j) for j in range(n_gpu * i, n_gpu * (i + 1))]
    res = ray.get(futures)
    ray.shutdown()
    res_all.append(res)
    arr_res_all = np.vstack(res_all)
    coverage = (arr_res_all[:, 0] < V_true) * (arr_res_all[:, 1] > V_true)
    mean_width = mean(arr_res_all[:, 1] - arr_res_all[:, 0])
    printR("time cost for {}-th rep = {:.1f} mins with coverage = {} and mean_width = {}".format(i, (now() - a) / 60
                                                                                                 , sum(coverage) / len(coverage)
                                                                                                , mean_width))
res_this_setting = {"CIs" : arr_res_all
                   , "coverage" : coverage
                   , "mean_width" : mean_width}
setting = "coindice_" + "cartpole_" + "alpha_" + str(alpha_CI) + "N_" + str(N) + "tau_" + str(tau) 
setting = setting + "algae_alpha_" + str(algae_alpha) + "lr_" + str(lr)  + "bs_" + str(batch_size)
path = "res/" + setting
printR(path)
dump(res_this_setting, path)

2020-11-05 18:06:03,402	INFO services.py:1166 -- View the Ray dashboard at http://127.0.0.1:8265
2020-11-05 18:06:45,719	WARNING worker.py:1072 -- Warning: The remote function __main__.one_seed1 has size 159391908 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.


time cost for 0-th rep = 3.2 mins with coverage = 0.125 and mean_width = 0.1444087028503418


2020-11-05 18:09:14,737	INFO services.py:1166 -- View the Ray dashboard at http://127.0.0.1:8265
2020-11-05 18:09:58,054	WARNING worker.py:1072 -- Warning: The remote function __main__.one_seed1 has size 159391908 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.


time cost for 1-th rep = 6.7 mins with coverage = 0.125 and mean_width = 0.14804387092590332


2020-11-05 18:12:47,778	INFO services.py:1166 -- View the Ray dashboard at http://127.0.0.1:8265
2020-11-05 18:13:31,170	WARNING worker.py:1072 -- Warning: The remote function __main__.one_seed1 has size 159391908 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.


...

time cost for 21-th rep = 76.9 mins with coverage = 0.19886363636363635 and mean_width = 0.15715433657169342


2020-11-05 19:23:00,508	INFO services.py:1166 -- View the Ray dashboard at http://127.0.0.1:8265
2020-11-05 19:23:45,010	WARNING worker.py:1072 -- Warning: The remote function __main__.one_seed1 has size 159391908 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.


time cost for 22-th rep = 80.4 mins with coverage = 0.20108695652173914 and mean_width = 0.15698908269405365


2020-11-05 19:26:30,059	INFO services.py:1166 -- View the Ray dashboard at http://127.0.0.1:8265
2020-11-05 19:27:13,090	WARNING worker.py:1072 -- Warning: The remote function __main__.one_seed1 has size 159391908 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.


time cost for 23-th rep = 83.8 mins with coverage = 0.203125 and mean_width = 0.15753501653671265
res/coindice_cartpole_alpha_0.9N_10tau_0.3algae_alpha_0.05lr_0.005bs_32


In [8]:
rep = 200
###########
lr = 0.005
batch_size = 32
###########
algae_alpha = 0.05
alpha_CI = 0.95

setting = "coindice_" + "cartpole_" + "alpha_" + str(alpha_CI) + "N_" + str(N) + "tau_" + str(tau) 
setting = setting + "algae_alpha_" + str(algae_alpha) + "lr_" + str(lr)  + "bs_" + str(batch_size)
path = "res/" + setting
printR(path)

ray.shutdown()
a = now()
res_all = []
for i in range(rep // n_gpu):
    @ray.remote(num_gpus=1)
    def one_seed1(seed):
        gpu_idx = seed % n_gpu
        pi1 = DQN.DQN_gym(num_states = 4, num_actions = 2
                          , hidden_units = [256, 256], gamma = gamma
                          , gpu_number = gpu_idx)
        pi1.model.load_weights(tp_path)
        dataset = converter.convertDataset(trajs_train_resp[seed]
                                      , min_A = 0, max_A = 1
                                       , min_S = -5, max_S = 5)
        CI = run_coin.coindice(dataset = dataset, pi1 = pi1, N = N, T = T, gamma = gamma
                      , hidden_dims = (64, ), gpu_number = gpu_idx
                 , nu_learning_rate = lr, zeta_learning_rate = lr, weight_learning_rate = lr ### ????
                 , nu_regularizer = 1, zeta_regularizer = 1 # NOT SENSITIVE
                 , num_steps = 2000, batch_size = batch_size, alpha_CI = alpha_CI
                 , print_loss = 0, print_freq = 50, print_prog = 0
                 , primal_form = True, f_exponent = 1.5, algae_alpha = algae_alpha)
        return CI
    ray.init()
    ###########
    futures = [one_seed1.remote(j) for j in range(n_gpu * i, n_gpu * (i + 1))]
    res = ray.get(futures)
    ray.shutdown()
    res_all.append(res)
    arr_res_all = np.vstack(res_all)
    coverage = (arr_res_all[:, 0] < V_true) * (arr_res_all[:, 1] > V_true)
    mean_width = mean(arr_res_all[:, 1] - arr_res_all[:, 0])
    printR("time cost for {}-th rep = {:.1f} mins with coverage = {} and mean_width = {}".format(i, (now() - a) / 60
                                                                                                 , sum(coverage) / len(coverage)
                                                                                                , mean_width))
res_this_setting = {"CIs" : arr_res_all
                   , "coverage" : coverage
                   , "mean_width" : mean_width}
dump(res_this_setting, path)

res/coindice_cartpole_alpha_0.95N_10tau_0.3algae_alpha_0.05lr_0.005bs_32


2020-11-05 23:20:39,451	INFO services.py:1166 -- View the Ray dashboard at http://127.0.0.1:8265
2020-11-05 23:21:24,083	WARNING worker.py:1072 -- Warning: The remote function __main__.one_seed1 has size 159391908 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.


time cost for 0-th rep = 3.2 mins with coverage = 0.125 and mean_width = 0.1735401153564453


2020-11-05 23:23:54,072	INFO services.py:1166 -- View the Ray dashboard at http://127.0.0.1:8265
2020-11-05 23:24:37,857	WARNING worker.py:1072 -- Warning: The remote function __main__.one_seed1 has size 159391908 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.


time cost for 1-th rep = 6.8 mins with coverage = 0.25 and mean_width = 0.18036293983459473


...

time cost for 21-th rep = 77.2 mins with coverage = 0.23295454545454544 and mean_width = 0.18930478394031525


2020-11-06 00:37:49,237	INFO services.py:1166 -- View the Ray dashboard at http://127.0.0.1:8265
2020-11-06 00:38:33,237	WARNING worker.py:1072 -- Warning: The remote function __main__.one_seed1 has size 159391908 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.


time cost for 22-th rep = 80.7 mins with coverage = 0.23369565217391305 and mean_width = 0.1891259104013443


2020-11-06 00:41:20,234	INFO services.py:1166 -- View the Ray dashboard at http://127.0.0.1:8265
2020-11-06 00:42:04,251	WARNING worker.py:1072 -- Warning: The remote function __main__.one_seed1 has size 159391908 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.


time cost for 23-th rep = 84.1 mins with coverage = 0.234375 and mean_width = 0.18931014835834503
